In [3]:
import numpy as np
import pandas as pd

In [4]:
attempts_df = pd.read_csv("TextBanking_Efforts.xlsx - Attempts.csv")
contact_df = pd.read_csv("TextBanking_Efforts.xlsx - Contact.csv")

# clean -- values
for column in attempts_df.columns:
    attempts_df = attempts_df[(attempts_df[column] != "--")]
for column in contact_df.columns:
    contact_df = contact_df[(contact_df[column] != "--")]

In [5]:
attempts_df.columns

Index(['timestamp', 'message_direction', 'Voter ID'], dtype='object')

In [6]:
contact_df.columns

Index(['Question Sent', 'question_Response_type', 'response to question',
       'updated_at', 'Voter ID'],
      dtype='object')

In [7]:
# dictionary mapping {Question : {available response : count}}
question_response = {}
total_count = 0

# initialize questions
for question in contact_df["Question Sent"].unique():
    question_response[question] = {}

# initialize available responses per question
for row_idx in range(0, contact_df.shape[0]):
    # 'Question Sent' is col 0, 'response to question' is col 2
    question = contact_df.iloc[row_idx, 0]
    response = contact_df.iloc[row_idx, 2]
    if response not in question_response[question]:
        question_response[question][response] = 1
        total_count += 1
    else:
        question_response[question][response] += 1
        total_count += 1

print(contact_df["Voter ID"].unique().shape[0])
question_response

419


{'Planning to vote?': {'No': 44, 'Yes': 200},
 'Do you have the proper ID to vote?': {'Yes': 49, 'No': 2, 'Unsure': 3},
 'Texting correct person': {'Contacted': 142,
  'Refused': 53,
  'Wrong Number': 111,
  'Spanish': 2,
  'Other Language': 1},
 'Texting the Right Person': {'Wrong Number': 8}}

In [8]:
# Define [successful contact] as someone who was contacted correctly and communicated with correctly
# Success at getting a response from reaching out, ie contact success rate
all_contacts_df = pd.DataFrame({"Voter ID": attempts_df["Voter ID"].unique()})
all_contacts_df["Contact Success"] = 0 # changed to 1 later if success
all_contacts_df = all_contacts_df.set_index('Voter ID')
all_contacts_df = all_contacts_df.sort_values(by=["Voter ID"])
contact_df = contact_df.sort_values(by=["Voter ID"])

# Voter ID is column 4 in contact_df
prev_voterID = contact_df.iloc[0, 4]
responses = {}
success = False # list of questions this person was asked over text

for row_idx in range(0, contact_df.shape[0]):
    curr_voterID = contact_df.iloc[row_idx, 4]
    # if new person, analyze the boolean and reset boolean
    if curr_voterID != prev_voterID:
        # analyze previous voterID. If success, change in all_contacts_df
        # various booleans
        not_plan_to_vote = 'Planning to vote?' in responses and responses['Planning to vote?']=='No'
        no_proper_ID = 'Do you have the proper ID to vote?' in responses and responses['Do you have the proper ID to vote?']!="Yes"
        not_correct_person = 'Texting correct person' in responses and responses['Texting correct person']!='Contacted'
        not_right_person = 'Texting the Right Person' in responses and responses['Texting the Right Person'] == 'Wrong Number'
        if not_correct_person or not_right_person:
            success = False
        if success:
            all_contacts_df.loc[prev_voterID, "Contact Success"] = 1
        success = True # default boolean to False
        responses = {} # reset responses

    prev_voterID = curr_voterID

all_contacts_df


,Contact Success
Voter ID,
1000205011,0
1000433225,0
1000746850,0
1001614539,0
1004309873,0
...,...
2193193194,0
2193193564,0
2193194892,0


In [9]:
# sanity check to make sure all voter ids in contacts_df are also in all_contacts_df
count = 0
for voter_id in contact_df["Voter ID"].unique():
    if voter_id not in all_contacts_df.index:
        count += 1
count

0

In [10]:
# sanity check for previous block:
num_success = 0
for row_idx in range(0, all_contacts_df.shape[0]):
    if all_contacts_df.iloc[row_idx, 0] == 1:
        num_success += 1
num_success

417

In [11]:
# get final percentage of successfully contacted people and a df of just the people that were successfully contacted
successful_contact_df = all_contacts_df[all_contacts_df["Contact Success"] == 1]
print("overall success rate = ", num_success / all_contacts_df.shape[0])
successful_contact_df

overall success rate =  0.0410150486869283


,Contact Success
Voter ID,
1041564212,1
1043427395,1
1061260847,1
1090798474,1
1102265724,1
...,...
2187464508,1
2188489848,1
2188895084,1


In [13]:
#Money Spending
totalMoneySpent = all_contacts_df.shape[0] * 0.06
moneyLost = (1-num_success / all_contacts_df.shape[0]) * all_contacts_df.shape[0] * 0.06
moneyWellSpent = num_success / all_contacts_df.shape[0] * all_contacts_df.shape[0] * 0.06


print("total money spent = ", totalMoneySpent)
print("money lost = ", moneyLost)
print("money well spent = ", moneyWellSpent)
success_by_location = {}

nov_votes_id = pd.read_csv("../cleaned_nov_votes.csv")["Voter ID"].tolist()
successful_contact_id = successful_contact_df.index.tolist()
print(successful_contact_id)
#Voter turn out
countVoterTurnedOut = 0
print("Voters we texted successfully (got response):", len(successful_contact_id))
print("Nov voters who turned out:", len(nov_votes_id))

for voterId in successful_contact_id:
   if (int(voterId) in nov_votes_id):
       countVoterTurnedOut += 1

print("Voters we contacted who turned out: ", countVoterTurnedOut)

costToGetOnePersonToVote = totalMoneySpent/countVoterTurnedOut

print("Cost to get One Person To Vote: $", costToGetOnePersonToVote)

total money spent =  610.02
money lost =  585.0
money well spent =  25.02
['1041564212', '1043427395', '1061260847', '1090798474', '1102265724', '1102280756', '1102288308', '1102288536', '1102298111', '1102311807', '1102319268', '1102321583', '1102334131', '1102344612', '1102360166', '1102365946', '1102381918', '1102620312', '1102720198', '1102751899', '1102752620', '1102816366', '1103032468', '1103045569', '1103049503', '1103057439', '1103069179', '1103073300', '1103089342', '1103127098', '1103138424', '1103153910', '1103154991', '1103184488', '1103201280', '1103258780', '1103577879', '1103719255', '1103942287', '1104208603', '1104248838', '1105733695', '1105752187', '1106105154', '1106126760', '1106633023', '1107270162', '1107676492', '1108045610', '1108782112', '1108798354', '1108806210', '1108812362', '1108858411', '1108858843', '1108866721', '1108882899', '1109169518', '1109243921', '1109255465', '1110122288', '1110198951', '1110263301', '1110277717', '1110286460', '1110411952', '